# Faster R-CNN

https://arxiv.org/pdf/1506.01497.pdf

![Figure 2](faster_r_cnn.png)

3 FASTER R-CNN

Our object detection system, called Faster R-CNN, is composed of two modules. The first module is a deep fully convolutional network that proposes regions, and the second module is the Fast R-CNN detector [2] that uses the proposed regions. The entire system is a single, unified network for object detection (Figure 2). Using the recently popular terminology of neural networks with ‘attention’ [31] mechanisms, the RPN module tells the Fast R-CNN module where to look. In Section 3.1 we introduce the designs and properties of the network for region proposal. In Section 3.2 we develop algorithms for training both modules with features shared.

我们的目标检测系统叫Faster R-CNN。它由两个模块组成。第一个模块是一个深度全卷积网络用来提供候选区域(RPN)。另一个模块是这些区域的使用者，Fast R-CNN检测器。整个系统是一个统一的网络。RPN告诉Fast R-CNN应该看哪些位置。3.1介绍“区域建议”的设计。3.2实现了共享两个模块属性的训练算法。

3.1 Region Proposal Networks

A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.3 We model this process with a fully convolutional network [7], which we describe in this section. Because our ultimate goal is to share computation with a Fast R-CNN object detection network [2], we assume that both nets share a common set of convolutional layers. In our experiments, we investigate the Zeiler and Fergus model [32] (ZF), which has 5 shareable convolutional layers and the Simonyan and Zisserman model [3] (VGG-16), which has 13 shareable convolutional layers.

输入一张任意尺寸的图片给 RPN 会得到一组候选边框，每一个框都关联一个物体分值。我们使用一个FCN来完成这个过程。因为我们的终极目标是要和Fast R-CNN网络共享计算，我们假设这两个网络使用同样的一组卷积层。在试验中，我们调研了ZF模型，它有5个共享的卷积层。SZ模型有13个共享的卷积层。

To generate region proposals, we slide a small network over the convolutional feature map output by the last shared convolutional layer. This small network takes as input an n × n spatial window of the input convolutional feature map. Each sliding window is mapped to a lower-dimensional feature (256-d for ZF and 512-d for VGG, with ReLU [33] following). This feature is fed into two sibling fully-connected layers—a box-regression layer (reg) and a box-classification layer (cls). We use n = 3 in this paper, noting that the effective receptive field on the input image is large (171 and 228 pixels for ZF and VGG, respectively). This mini-network is illustrated at a single position in Figure 3 (left). Note that because the mini-network operates in a sliding-window fashion, the fully-connected layers are shared across all spatial locations. This architecture is naturally implemented with an n×n convolutional layer followed by two sibling 1 × 1 convolutional layers (for reg and cls, respectively).

为了生成候选的区域，我们在共享网络的最后一层的输出的卷积特征图上用一个小网络来滑动。这个小网络的输入是一个空间大小n x n的窗口。每个窗口关联到一个低维的属性（ZF-256维，vgg-512维后跟relu）。这个属性输出到两个并列的fc层：1个框回归层（reg）和一个框分类层（cls）。这篇文章中我们使用n=3，在输入图片上有效的卷积域很大（ZF有171个像素，VGG有228）。这个小网络在图3中表示为一个位置。因为小网络通过滑动的方式来工作，fc层就共享了所有位置的空间信息。这个模式自然地实现成一个n x n的卷积层后跟两个并列的1 x 1的卷积层（分别对应于reg和cls）。

![Figure 3](faster_r_cnn_rpn.png)

3.1.1 Anchors

At each sliding-window location, we simultaneously predict multiple region proposals, where the number of maximum possible proposals for each location is denoted as k. So the reg layer has 4k outputs encoding the coordinates of k boxes, and the cls layer outputs 2k scores that estimate probability of object or not object for each proposal4. The k proposals are parameterized relative to k reference boxes, which we call anchors. An anchor is centered at the sliding window in question, and is associated with a scale and aspect ratio (Figure 3, left). By default we use 3 scales and 3 aspect ratios, yielding k = 9 anchors at each sliding position. For a convolutional feature map of a size W × H (typically ∼2,400), there are W H k anchors in total.

在滑动窗口的每一个位置，我们同时预测最多k个区域。所以reg层就有4k个输出来代表边框的坐标。cls层有2k个输出用来代表某个物体出现和不出现的概率。我们把这k个建议叫做anchors（锚）。每一个anchor在滑动窗口中居中，并且和一个尺度和一个纵横比关联。默认我们使用3个尺度和3个纵横比，也就是每个滑动区域产生k = 9个anchors。对于一个W x H（一般为2400）的卷积特征图，一共有W H K个锚点。

3. “Region” is a generic term and in this paper we only consider rectangular regions, as is common for many methods (e.g., [27], [4], [6]). “Objectness” measures membership to a set of object classes vs. background.

    - “Region” 是一个通用的术语，在本文中我们只讨论矩形区域。
    - “Objectness” 描述是否是一个对象还是背景

4. For simplicity we implement the cls layer as a two-class softmax layer. Alternatively, one may use logistic regression to produce k scores.

    为了简单，我们用2分类softmax层来实现cls层。或者使用logistic回归来预测k个值。

Translation-Invariant Anchors

An important property of our approach is that it is translation invariant, both in terms of the anchors and the functions that compute proposals relative to the anchors. If one translates an object in an image, the proposal should translate and the same function should be able to predict the proposal in either location. This translation-invariant property is guaranteed by our method5. As a comparison, the MultiBox method [27] uses k-means to generate 800 anchors, which are not translation invariant. So MultiBox does not guarantee that the same proposal is generated if an object is translated.

我们的方法有一个重要的特征就是与变换无关。无论是anchor还是和anchor相关的计算函数。如果原图片上发生了一个变换，候选也会发生变换使候选区域仍能产生。相比于MultiBox方法，它使用k均值来生成800个锚点，就不具有这种属性。它不能保证在图片发生了变换以后，候选区仍然能给产生。

The translation-invariant property also reduces the model size. MultiBox has a (4 + 1) × 800-dimensional fully-connected output layer, whereas our method has a (4 + 2) × 9-dimensional convolutional output layer in the case of k = 9 anchors. As a result, our output layer has 2.8 × 104 parameters (512 × (4 + 2) × 9 for VGG-16), two orders of magnitude fewer than MultiBox’s output layer that has 6.1 × 106 parameters (1536 × (4 + 1) × 800 for GoogleNet [34] in MultiBox [27]). If considering the feature projection layers, our proposal layers still have an order of magnitude fewer parameters than MultiBox6. We expect our method to have less risk of overfitting on small datasets, like PASCAL VOC.

这种变换无关属性还减小了模型的尺寸。MultiBox有`(4+1) * 800`维fc层，而我们的方法只有`(4+2) * 9`维的conv层作为输出。因此我们的输出层有 $2.8 \times 10^4$个参数（`512 * (4 + 2) * 9` VGG的情况下），比MultiBox的输出层少了两个数量级，它有$6.1 \times 10^6$个参数（`1536 * (4 + 1) * 800` 使用GooleNet的情况）。如果考虑特征映射层的情况下，我们的方法比MultiBox还能下降几个数量级。从而我们期望减小在小数据集上过拟合，例如PASCAL VOC。

5. As is the case of FCNs [7], our network is translation invariant up to the network’s total stride.

6. Considering the feature projection layers, our proposal layers’ parameter count is 3×3×512×512+512×6×9 = 2.4×106; MultiBox’s proposal layers’ parameter count is 7 × 7 × (64 + 96 + 64+64)×1536+1536×5×800 = 27×106.


Multi-Scale Anchors as Regression References

Our design of anchors presents a novel scheme for addressing multiple scales (and aspect ratios). As shown in Figure 1, there have been two popular ways for multi-scale predictions. The first way is based on image/feature pyramids, e.g., in DPM [8] and CNN- based methods [9], [1], [2]. The images are resized at multiple scales, and feature maps (HOG [8] or deep convolutional features [9], [1], [2]) are computed for each scale (Figure 1(a)). This way is often useful but is time-consuming. The second way is to use sliding windows of multiple scales (and/or aspect ratios) on the feature maps. For example, in DPM [8], models of different aspect ratios are trained separately using different filter sizes (such as 5×7 and 7×5). If this way is used to address multiple scales, it can be thought of as a “pyramid of filters” (Figure 1(b)). The second way is usually adopted jointly with the first way [8].

这种新颖的锚点的设计是为了解决多尺度（多高宽比）的问题。如图1所示，有两种主流的多尺度预测方法。（图片多尺度，卷积核多尺度）。第一种方法基于图片/特征金字塔，例如DPM和基于CNN的方法。图片被变换成多种大小后，对每个大小的图片进行特征提取。这种方法很有用，但是耗时。第二种方法是在特征图上使用不同大小的滑动窗口。例如，DPM使用不同的滤波器（5x7和7x5）来分开训练不同的模型。用这种方法来处理多尺度的问题，可以看成“滤波器金字塔”。两种方法经常结合着使用。

As a comparison, our anchor-based method is built on a pyramid of anchors, which is more cost-efficient. Our method classifies and regresses bounding boxes with reference to anchor boxes of multiple scales and aspect ratios. It only relies on images and feature maps of a single scale, and uses filters (sliding windows on the feature map) of a single size. We show by experiments the effects of this scheme for addressing multiple scales and sizes (Table 8).

相比之下，我们基于锚点的方法是建立在锚点金字塔上，更加高效。使用不同尺度的锚点来做分类和边框回归。只需要一个尺度的图片和一个尺度的多个卷积核。在表格8中显示了这种方法在不同尺度大小上的效果。

Because of this multi-scale design based on anchors, we can simply use the convolutional features computed on a single-scale image, as is also done by the Fast R-CNN detector [2]. The design of multi-scale anchors is a key component for sharing features without extra cost for addressing scales.

3.1.2 Loss Function

For training RPNs, we assign a binary class label (of being an object or not) to each anchor. We assign a positive label to two kinds of anchors: (i) the anchor/anchors with the highest Intersection-over-Union (IoU) overlap with a ground-truth box, or (ii) an anchor that has an IoU overlap higher than 0.7 with any ground-truth box. Note that a single ground-truth box may assign positive labels to multiple anchors. Usually the second condition is sufficient to determine the positive samples; but we still adopt the first condition for the reason that in some rare cases the second condition may find no positive sample. We assign a negative label to a non-positive anchor if its IoU ratio is lower than 0.3 for all ground-truth boxes. Anchors that are neither positive nor negative do not contribute to the training objective.

为了训练RPN，我们为每个锚点指定了一个二值的标签。正标签给两种锚点：1）有最大IoU的锚点 或者 2）IoU大于0.7的锚点。一个真值框可能对应多个正标签的锚点。一般情况下第二种条件就足以找到真样本，但在极端情况下第一种条件来兼容。负标签分配给IoU小于0.3的锚点。非正或者非负的锚点不参与到训练中来。

With these definitions, we minimize an objective function following the multi-task loss in Fast R-CNN [2]. Our loss function for an image is defined as:

有了这些定义，在Fast R-CNN中我们最小化下面这个多目标的损失函数。

$$
L(\{p_i\}, \{t_i\}) = \frac{1}{N_{cls}} \sum_i L_{cls} (p_i, p_i^*) \\
+ \lambda \frac{1}{N_{reg}} \sum_i p_i^* L_{reg} (t_i, t_i^*)
$$

Here, i is the index of an anchor in a mini-batch and pi is the predicted probability of anchor i being an object. The ground-truth label p∗i is 1 if the anchor is positive, and is 0 if the anchor is negative. ti is a vector representing the 4 parameterized coordinates of the predicted bounding box, and t∗i is that of the ground-truth box associated with a positive anchor.

这里的`i`是某个锚点的下标，$p_i$是这个锚点是一个物体的概率。真实值$p_i^*$在正锚点的时候是1，负锚点时为0. $t_i$是4维向量表示的边框，$t_i^*$是真实的边框。

The classification loss Lcls is log loss over two classes (object vs. not object). For the regression loss, we use Lreg (ti, t∗i ) = R(ti − t∗i ) where R is the robust loss function (smooth L1) defined in [2]. The term p∗i Lreg means the regression loss is activated only for positive anchors (p∗i = 1) and is disabled otherwise (p∗i = 0). The outputs of the cls and reg layers consist of {pi} and {ti} respectively.

分类误差$L_{cls}$是xxx。回归误差

The two terms are normalized by Ncls and Nreg and weighted by a balancing parameter λ. In our current implementation (as in the released code), the cls term in Eqn.(1) is normalized by the mini-batch size (i.e., Ncls = 256) and the reg term is normalized by the number of anchor locations (i.e., Nreg ∼ 2, 400). By default we set λ = 10, and thus both cls and reg terms are roughly equally weighted. We show by experiments that the results are insensitive to the values of λ in a wide range (Table 9). We also note that the normalization as above is not required and could be simplified.

For bounding box regression, we adopt the param- eterizations of the 4 coordinates following [5]:

tx =(x−xa)/wa, ty =(y−ya)/ha,
tw = log(w/wa), th = log(h/ha), (2)
t∗x = (x∗ − xa)/wa, t∗y = (y∗ − ya)/ha, t∗w = log(w∗/wa), t∗h = log(h∗/ha),

where x, y, w, and h denote the box’s center coordi- nates and its width and height. Variables x, xa, and x∗ are for the predicted box, anchor box, and ground- truth box respectively (likewise for y, w, h). This can
be thought of as bounding-box regression from an anchor box to a nearby ground-truth box.

Nevertheless, our method achieves bounding-box regression by a different manner from previous RoI- based (Region of Interest) methods [1], [2]. In [1], [2], bounding-box regression is performed on features pooled from arbitrarily sized RoIs, and the regression weights are shared by all region sizes. In our formula- tion, the features used for regression are of the same spatial size (3 × 3) on the feature maps. To account for varying sizes, a set of k bounding-box regressors are learned. Each regressor is responsible for one scale and one aspect ratio, and the k regressors do not share weights. As such, it is still possible to predict boxes of various sizes even though the features are of a fixed size/scale, thanks to the design of anchors.


3.1.3 Training RPNs

The RPN can be trained end-to-end by back- propagation and stochastic gradient descent (SGD) [35]. We follow the “image-centric” sampling strategy from [2] to train this network. Each mini-batch arises from a single image that contains many positive and negative example anchors. It is possible to optimize for the loss functions of all anchors, but this will bias towards negative samples as they are dominate. Instead, we randomly sample 256 anchors in an image to compute the loss function of a mini-batch, where the sampled positive and negative anchors have a ratio of up to 1:1. If there are fewer than 128 positive samples in an image, we pad the mini-batch with negative ones.

We randomly initialize all new layers by drawing weights from a zero-mean Gaussian distribution with standard deviation 0.01. All other layers (i.e., the shared convolutional layers) are initialized by pre- training a model for ImageNet classification [36], as is standard practice [5]. We tune all layers of the ZF net, and conv3 1 and up for the VGG net to conserve memory [2]. We use a learning rate of 0.001 for 60k mini-batches, and 0.0001 for the next 20k mini-batches on the PASCAL VOC dataset. We use a momentum of 0.9 and a weight decay of 0.0005 [37]. Our implementation uses Caffe [38].

3.2 Sharing Features for RPN and Fast R-CNN

Thus far we have described how to train a network for region proposal generation, without considering the region-based object detection CNN that will utilize these proposals. For the detection network, we adopt Fast R-CNN [2]. Next we describe algorithms that learn a unified network composed of RPN and Fast R-CNN with shared convolutional layers (Figure 2).

Both RPN and Fast R-CNN, trained independently, will modify their convolutional layers in different ways. We therefore need to develop a technique that allows for sharing convolutional layers between the